In [1]:
from utils_demo import *
from loguru import logger
from unittest.mock import patch

In [2]:
fig = plot_detected_phases(jobid=3911)
fig.show()
 

c:\Users\a770398\AppData\Local\Continuum\anaconda3\envs\iosea-venv\lib\site-packages\pwlf\pwlf.py:1113: RuntimeWarning:

divide by zero encountered in true_divide

c:\Users\a770398\AppData\Local\Continuum\anaconda3\envs\iosea-venv\lib\site-packages\pwlf\pwlf.py:1113: RuntimeWarning:

invalid value encountered in true_divide

c:\Users\a770398\AppData\Local\Continuum\anaconda3\envs\iosea-venv\lib\site-packages\pwlf\pwlf.py:1118: RuntimeWarning:

invalid value encountered in multiply



labels=[0 2 2 0 0 0 0 0 2 2 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
starting points = [1, 8, 14]
ending points = [3, 10, 16]
labels=[0 2 2 0 0 0 0 0 2 2 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
starting points = [1, 8, 14]
ending points = [3, 10, 16]


In [3]:
logger.remove()
def decompose_ioi_job(jobid):
    with patch.object(ComplexDecomposer, 'get_job_timeseries') as mock_get_timeseries:
        with patch.object(Configuration, 'get_kc_token') as mock_get_kc_token:
            mock_get_timeseries.return_value = get_job_timeseries_from_file(job_id=jobid)
            mock_get_kc_token.return_value = 'token'
            # init the job decomposer
            cd = ComplexDecomposer()
            return cd
cd = decompose_ioi_job(jobid=3911)
compute, reads, read_bw, writes, write_bw = cd.get_job_representation()
# This is the app encoding representation for Execution Simulator
print(f"compute={compute}, reads={reads}, read_bw={read_bw}")
print(f"compute={compute}, writes={writes}, write_bw={write_bw}")
# Normalize signals
timestamps = (cd.timestamps.flatten() - cd.timestamps.flatten()[0])/5
original_read =  cd.read_signal.flatten()/1e6
original_write = cd.write_signal.flatten()/1e6



labels=[0 2 2 0 0 0 0 0 2 2 0 0 0 3 2 2 0 0 0 0 0 3 3 0 3 3 3 3 3 0 0 3 3 0 0 0 0
 0 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 0 0
 0 0 0 0 0 0 0 0 3 3 3 3 3 3 0 0 0 0 0 3 3 3 0]
starting points = [1, 8, 13, 14, 21, 24, 31, 44, 59, 82, 93]
ending points = [3, 10, 14, 16, 23, 29, 33, 47, 72, 88, 96]
labels=[0 2 2 0 0 0 0 0 2 2 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
starting points = [1, 8, 14]
ending points = [3, 10, 16]
labels=[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 0 2 2 3 3 3 0 4 2 2 0 0 0 0
 0 0 0 0 0 0 0 2 2 2 4 0 0 0 0 0 0 0 0 0 0 0 2 2 2 3 2 2 2 2 2 3 3 2 2 0 0
 0 0 0 0 0 0 0 0 2 2 3 3 2 4 0 0 0 0 4 2 2 2 0]
starting points = [21, 24, 26, 30, 31, 44, 47, 59, 62, 63, 68, 70, 82, 84, 86, 87, 92, 93]
ending points = [23, 26, 29, 31, 33, 47, 48, 62, 63, 68, 70, 72, 84, 86, 87, 88, 93, 96]
compute=[0, 1, 7, 11, 12, 1

Now we obtain a representation of the job that "removes" the performance of the storage tier:
- Events (timestamps) = [0, 1, 7, 11, 12, 18, 20, 23, 35, 48, 59, 65, 66]
- Reads (volumes in GB) = [0.0, 23.39, 31.69, 0.0, 28.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
- Writes (volumes in GB) = [0.0, 0.0, 0.0, 0.0, 0.0, 2.07, 9.79, 3.51, 4.32, 21.23, 10.67, 4.58, 0.0]

In [4]:
print(compute)
print(list(map(lambda x: round(x/1e9, 2), reads)))
print(list(map(lambda x: round(x/1e9, 2), writes)))


[0, 1, 7, 11, 12, 18, 20, 23, 35, 48, 59, 65, 66]
[0.0, 23.39, 31.69, 0.0, 28.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 2.07, 9.79, 3.51, 4.32, 21.23, 10.67, 4.58, 0.0]


Signal reconstruction from the representation:
- we reuse measured bandwidths for both read and write signal
- we feed the Simulator with this data, instead of providing a Storage Performance Model: bw = f(IO_size, IO_pattern, ...)

In [10]:
# Apply same BW as measured by IOI
read_bw = list(map(lambda x: x/1e6, read_bw))
write_bw = list(map(lambda x: x/1e6, write_bw))
io_bw = list(map(lambda x, y: x + y, read_bw, write_bw))

Inputting representation in App Simulator


In [11]:
sim_time, sim_read_bw, sim_write_bw = simulate_app(compute, reads, writes, io_bw, app_name="3911")


Plotting Simulation versus Origial (IOI) signals

In [15]:
fig = display_original_sim_signals((sim_time, sim_read_bw, sim_write_bw),
                                   (timestamps, original_read, original_write),
                                   width=1400, height=900)
fig.show()

- Until now, we just replayed the representation into the simulator.
- But the aim of the Simulator is to predict the expected execution time for a different storage tier.
- The Simualtor will accept later the performance model for a tier bw = f(IO_size, IO_pattern,...)
- For now we will __mock__ the model with a hardcoded io_bandwidth

In [16]:
tier_mean_read_bw = 5e3
tier_mean_write_bw = 1e3
io_bw = list(map(lambda x, y: tier_mean_read_bw if x else 0 + tier_mean_write_bw if y else 0, read_bw, write_bw))

And rerun the simulation

In [17]:
fig = display_original_sim_signals(simulate_app(compute, reads, writes, io_bw, app_name="3911"),
                                   (timestamps, original_read, original_write),
                                   width=1400, height=900)
fig.show()

The expected duration of the application is 129*5s = 645 seconds with this storage instead of 97*5s = 485 seconds.

Conclusion and further improvements:
- fix bugs on the AppDecompser 0.1
- update simulator to accept phases having both read and write I/O
- build a Performance Model with bandwidths measured directly from the storage tier
- progressing toward simulating workflows having many jobs in parrallel and recommending placement for each step.